In [ ]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials


# Ensure the destination root directory exists
destination_root = '/content/drive/MyDrive/hwi_product'
if not os.path.exists(destination_root):
    os.makedirs(destination_root)

# Scope for Google Drive authentication
scope = ['https://www.googleapis.com/auth/drive']

# Initialize GoogleAuth
gauth = GoogleAuth()

# Function to download a file from Google Drive
def download_file(file, destination_folder):
    file_path = os.path.join(destination_folder, file['title'])
    file.GetContentFile(file_path)
    print(f"Downloaded: {file['title']} to {file_path}")

# Function to find a folder by name
def find_folder_by_name(drive, folder_name):
    folder_list = drive.ListFile({'q': f"title='{folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()
    if folder_list:
        return folder_list[0]
    return None

# Iterate through the years with corresponding service accounts
for index, start_year in enumerate(range(1960, 2024, 10), start=1):
    print(f"Index: {index}, Year: {start_year}")

    # Select the service account JSON file
    private_key_file = f'/content/drive/MyDrive/Colab Notebooks/gee/unicef-ccri{index}.json'

    # Create the folder name based on the start year
    folder_name = f'hwi_{start_year}s'

    # Create the local folder if it doesn't exist
    local_folder_path = os.path.join(destination_root, folder_name)
    if not os.path.exists(local_folder_path):
        os.mkdir(local_folder_path)

    # Authenticate to the Google Drive of the Service Account
    gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name(private_key_file, scopes=scope)
    drive = GoogleDrive(gauth)

    # Find the folder by name in Google Drive
    folder = find_folder_by_name(drive, folder_name)

    if folder:
        print(f"Found folder: {folder_name} with ID: {folder['id']}")

        # List files in the found folder
        file_list = drive.ListFile({'q': f"'{folder['id']}' in parents and trashed=false"}).GetList()
        for file in file_list:
            print('Title: %s, ID: %s' % (file['title'], file['id']))

            # Download each file to the local folder
            download_file(file, local_folder_path)
    else:
        print(f"Folder {folder_name} not found in Google Drive.")

# Note: Ensure to reset any state if necessary after the loop
